In [21]:
from selenium import webdriver
from time import sleep
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
import json
import pandas as pd

In [22]:
options = Options()
options.headless = False
options.add_argument("--window-size=1920,1080")
service = Service(executable_path='./chromedriver.exe')

In [23]:
news_dict = {'category':[], 'title': [], 'date': [], 'content': [], 'sources':[]}

# Crawling Process
1. get url about agricultural product
    - [RIce](http://agro.gov.vn/vn/xc9_Lua-gao.html)
    - [Coffee](http://agro.gov.vn/vn/xc2_Ca-phe.html)
2. from url we got from (1), access link to get title, date and content news
3. save to jsonl and csv

## Get Url

In [24]:
for i in range(128,141):
    browser = webdriver.Chrome(service=service, options=options)
    browser.get(f"https://agro.gov.vn/vn/p{i}_9Lua-gao.html")
    elements = browser.find_elements(By.CSS_SELECTOR, ".news [href]")
    links = [element.get_attribute('href') for element in elements]
    browser.quit()
    news_dict['sources'] += [*set(links[0:10])]

## Get Title, Content, Data

In [25]:
for i in range(len(news_dict['sources'])):
    browser = webdriver.Chrome(service=service, options=options)
    url = news_dict['sources'][i]
    browser.get(url)
    try:
        #Title
        element = browser.find_element(By.ID, "ctl00_maincontent_N_TIEUDE")
        news_dict['title'].append(element.text)
        #Date
        element = browser.find_element(By.ID, "ctl00_maincontent_N_NGAYTHANG")
        news_dict['date'].append(element.text)
        #Content
        element = browser.find_element(By.ID, "ctl00_maincontent_N_NOIDUNG")
        news_dict['content'].append(element.text)

    except:
        news_dict['title'].append(" ")
        news_dict['date'].append(" ")
        news_dict['content'].append(" ")
    browser.quit()

In [29]:
news_dict["category"] = ["rice"]*130

In [30]:
pd.DataFrame.from_dict(news_dict).to_csv('data/rice_news_128-141.csv')